Imports and setting environment vars

In [25]:
import os

import bs4
from langchain import hub
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.documents import Document
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langgraph.graph import START, StateGraph
from typing_extensions import List, TypedDict
from dotenv import load_dotenv

load_dotenv()

True

# Indexing

## Load Document(s)
Load an html webpage, and get only headers, titles, and contents(?).
We expect only one document `assert len(docs)==1`
We print the first 500 characters of the doc.

In [26]:
loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header")
        ),
    )
)
docs = loader.load()
assert len(docs)==1
print(f"Total characters:{len(docs[0].page_content)}")
print(docs[0].page_content[:500])

Total characters:43130


      LLM Powered Autonomous Agents
    
Date: June 23, 2023  |  Estimated Reading Time: 31 min  |  Author: Lilian Weng


Building agents with LLM (large language model) as its core controller is a cool concept. Several proof-of-concepts demos, such as AutoGPT, GPT-Engineer and BabyAGI, serve as inspiring examples. The potentiality of LLM extends beyond generating well-written copies, stories, essays and programs; it can be framed as a powerful general problem solver.
Agent System Overview#
In


## Split Documents
We'll now split the document into snippets of 1000 characters each.

In [27]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
all_splits = text_splitter.split_documents(docs)
print(f"Split blog post into {len(all_splits)} sub-documents.")

Split blog post into 66 sub-documents.


## Store Vectors
We use OPENAPI for embeddings.

In [28]:
openai_api_key = os.getenv("OPENAI_API_KEY")
from langchain_openai import OpenAIEmbeddings
embeddings = OpenAIEmbeddings(model="text-embedding-3-large", openai_api_key=openai_api_key)
from langchain_core.vectorstores import InMemoryVectorStore
vector_store = InMemoryVectorStore(embeddings)

In [29]:
document_ids=vector_store.add_documents(all_splits)
print(f"vector store now contains {len(document_ids)} vectors.")
print(document_ids[:3])

vector store now contains 66 vectors.
['274d6e00-a0b5-4f7e-afb0-78a515a63600', '805bfff6-8d27-4fe8-af2b-67f2362f97ad', '4507f4cc-2493-460a-9e38-b7323b59f249']


# Retrieval and Generation

In [42]:
from langchain import hub
import getpass
LANGSMITH_TRACING=True
LANGSMITH_ENDPOINT="https://api.smith.langchain.com"
LANGSMITH_API_KEY="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"
LANGSMITH_PROJECT="pr-abandoned-toenail-3"
OPENAI_API_KEY=os.getenv("OPENAI_API_KEY")
prompt = hub.pull("rlm/rag-prompt",api_key=LANGSMITH_API_KEY)
example_messages = prompt.invoke({"context":"(context goes here)", "question":"(question goes here)"}).to_messages()
assert len(example_messages)==1
print(example_messages[0].content)

You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.
Question: (question goes here) 
Context: (context goes here) 
Answer:


In [47]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model = "gpt-4o-mini")

In [53]:
class State(TypedDict):
    question: str
    context: List[Document]
    answer: str

def retrieve(state:State):
    retrieved_docs = vector_store.similarity_search(state["question"])
    return {"context": retrieved_docs}

def generate(state:State):
    docs_content="\n\n".join(doc.page_content for doc in state["context"])
    messages = prompt.invoke({"question":state["question"], "context":docs_content})
    response0 = llm.invoke(messages)
    return {"answer":response0.content}

In [54]:
graph_builder=StateGraph(State).add_sequence([retrieve,generate])

In [55]:
graph_builder.add_edge(START, "retrieve")
graph=graph_builder.compile()



In [56]:
response = graph.invoke({"question": "What is Task Decomposition?"})
print(response["answer"])

Task Decomposition is the process of breaking down complex tasks into smaller, manageable steps to facilitate easier execution and understanding. It can involve techniques like Chain of Thought (CoT) prompting, where a model is guided to think step-by-step, or Tree of Thoughts, which explores multiple reasoning possibilities at each step. This approach improves performance on intricate tasks by making them simpler and allows for clearer interpretation of the model's reasoning process.
